### Import libraries

In [1]:
import sqlalchemy as db
import pandas as pd

# Check SQLite version
print("SQLite version:\t", db.__version__)

SQLite version:	 1.4.37


### Connecting to a database

In [2]:
# Create a database engine 
engine = db.create_engine('sqlite:///sqlite_db_pythonsqlite.db')
insp = db.inspect(engine)
table_names = insp.get_table_names()
print("Table names: ", table_names) # Inspect table names

Table names:  ['Bookings', 'Facilities', 'Members']


### Explore tables

In [3]:
# Explore Bookings table
stmt = (
    """
    SELECT *
    FROM Bookings
    LIMIT 5;
    """)

with engine.connect() as conn:
    rs = conn.execute(stmt)
    bookings = pd.DataFrame(rs.fetchall())
    bookings.columns = rs.keys()
    
bookings # Print dataframe

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [4]:
# Explore Facilities table
stmt = (
    """
    SELECT *
    FROM Facilities
    LIMIT 5;
    """)

with engine.connect() as conn:
    rs = conn.execute(stmt)
    faci_df = pd.DataFrame(rs.fetchall())
    faci_df.columns = rs.keys()
    
faci_df # Print dataframe

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [5]:
# Explore Members table
stmt = (
    """
    SELECT *
    FROM Members
    LIMIT 5;
    """)

with engine.connect() as conn:
    rs = conn.execute(stmt)
    mem_df = pd.DataFrame(rs.fetchall())
    mem_df.columns = rs.keys()
    
mem_df # Print dataframe

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


### Part 2: SQLite

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [6]:
q10 = ("""
SELECT 
    name,
    SUM(revenue) AS total_revenue
FROM
    (SELECT
        f.name, 
        CASE 
            WHEN b.memid = 0 THEN b.slots * f.guestcost
        ELSE b.slots * f.membercost
        END AS revenue
    FROM Bookings AS b
        INNER JOIN Facilities AS f
            ON b.facid = f.facid
        INNER JOIN Members AS m
            ON b.memid = m.memid) AS subq
GROUP BY name
HAVING total_revenue < 1000
ORDER BY total_revenue;
""")

with engine.connect() as conn:
    rs = conn.execute(q10)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
df

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [7]:
q11 = ("""
SELECT 
	m.surname AS member_surname,
	m.firstname AS member_firstname,
	r.surname AS recommender_surname, 
	r.firstname AS recommender_firstname
FROM Members AS m
INNER JOIN Members AS r
	ON m.recommendedby = r.memid
WHERE m.recommendedby != ''
ORDER BY 1, 2;
""")

with engine.connect() as conn:
    rs = conn.execute(q11)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
df

,member_surname,member_firstname,recommender_surname,recommender_firstname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Genting,Matthew,Butters,Gerald
9,Hunt,John,Purview,Millicent


Q12: Find the facilities with their usage by member, but not guests.

In [8]:
q12 = ("""
SELECT 
	b.memid,
	m.surname || ', ' ||  m.firstname AS member_name,
	f.name AS facility,
	SUM(b.slots) AS slots
FROM Bookings AS b
INNER JOIN  Facilities AS f
	ON b.facid = f.facid
INNER JOIN Members AS m
	ON b.memid = m.memid
WHERE b.memid != 0
GROUP BY b.memid, f.name;
""")

with engine.connect() as conn:
    rs = conn.execute(q12)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
df

,memid,member_name,facility,slots
0,1,"Smith, Darren",Badminton Court,432
1,1,"Smith, Darren",Massage Room 1,58
2,1,"Smith, Darren",Pool Table,28
3,1,"Smith, Darren",Snooker Table,24
4,1,"Smith, Darren",Squash Court,30
...,...,...,...,...
197,35,"Hunt, John",Tennis Court 2,12
198,36,"Crumpet, Erica",Badminton Court,6
199,36,"Crumpet, Erica",Massage Room 1,4
200,36,"Crumpet, Erica",Table Tennis,4


Q13: Find the facilities usage by month, but not guests.

In [18]:
q13 = ("""
SELECT 
	(SELECT strftime('%m', b.starttime)) AS month,
	f.name AS facility,
	SUM(b.slots) AS slots
FROM Bookings AS b
INNER JOIN Facilities AS f
	ON b.facid = f.facid
GROUP BY facility, month
ORDER BY month, facility;
""")

with engine.connect() as conn:
    rs = conn.execute(q13)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
df

,month,facility,slots
0,07,Badminton Court,180
1,07,Massage Room 1,264
2,07,Massage Room 2,24
3,07,Pool Table,117
4,07,Snooker Table,156
5,07,Squash Court,164
6,07,Table Tennis,104
7,07,Tennis Court 1,270
8,07,Tennis Court 2,207
9,08,Badminton Court,459
